#Ahihiiii

In [8]:
%pip install pathlib
%pip install huggingface_hub
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [14]:
%pip install pathlib

Note: you may need to restart the kernel to use updated packages.


In [33]:
DEFAULT_COLAB_MOUNTED_FOLDER = "DATA"
PROJECT_NAME = "VIDEOMAE"

import os
from pathlib import Path

def make_dir(name: Path | str):
  if isinstance(name, str):
    dir_path = Path(name)
  if not dir_path.is_dir():
    dir_path = dir_path.parent
  dir_path.mkdir(parents=True, exist_ok=True)
  return Path(name).as_posix()

def get_resource(name: str) -> str:
  return make_dir(os.path.join(DEFAULT_COLAB_MOUNTED_FOLDER, PROJECT_NAME, name))

In [19]:
import shutil
from huggingface_hub import hf_hub_download


hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"
)
print(file_path)
shutil.copy(file_path, get_resource("UCF101_subset.tar.gz"))

C:\Users\ninhn\.cache\huggingface\hub\datasets--sayakpaul--ucf101-subset\snapshots\b9984b8d2a95e4a1879e1b071e9433858d0bc24a\UCF101_subset.tar.gz


'G:/CODE/VIDEOMAE/videomae-1/DATA/VIDEOMAE\\UCF101_subset.tar.gz'

In [26]:
%pip install wget

  Using cached wget-3.2.zip (10 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gzip (from versions: none)
ERROR: No matching distribution found for gzip


In [34]:
import tarfile

tarfile.open(get_resource("UCF101_subset.tar.gz")).extractall(get_resource("UCF101_subset"), filter='data')

DATA/VIDEOMAE/UCF101_subset.tar.gz
